In [3]:
import torch
print(torch.__version__)
print(torch.cuda.is_available()) 
print(torch.cuda.get_device_name(0))  


2.5.1
True
NVIDIA GeForce RTX 3090


### 1. 라이브러리 불러오기

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np


### 2. KLUE-RoBERTa 모델 & 토크나이저 로드

In [22]:
model_name = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3. NSMC 감정 데이터셋 로딩

In [23]:
dataset = load_dataset("nsmc", trust_remote_code=True)

### 4. 토큰화 처리

In [ ]:

def tokenize_function(examples):
    tokens = tokenizer(
        examples["document"],
        truncation=True,
        padding="max_length",  # ← 중요! 길이 통일
        max_length=256
    )
    tokens["labels"] = examples["label"]
    return tokens

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["document"]  # 원래 텍스트는 제거
)


Map: 100%|██████████| 50000/50000 [00:03<00:00, 13159.21 examples/s]


### 5. 정확도 평가 지표(evaluate 사용)

In [25]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### 6. 학습 설정 정의

In [26]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)


### 7. Trainer 생성 및 학습 실행

In [27]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267900,0.245908,0.901860
2,0.183800,0.258946,0.908440
3,0.153900,0.338521,0.909380


TrainOutput(global_step=28125, training_loss=0.21538819247775606, metrics={'train_runtime': 5466.6389, 'train_samples_per_second': 82.317, 'train_steps_per_second': 5.145, 'total_flos': 5.9199987456e+16, 'train_loss': 0.21538819247775606, 'epoch': 3.0})

### 8. 모델 저장

In [ ]:
model_path = "./klue-roberta-nsmc-finetuned"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('./klue-roberta-nsmc-finetuned\\tokenizer_config.json',
 './klue-roberta-nsmc-finetuned\\special_tokens_map.json',
 './klue-roberta-nsmc-finetuned\\vocab.txt',
 './klue-roberta-nsmc-finetuned\\added_tokens.json',
 './klue-roberta-nsmc-finetuned\\tokenizer.json')

### 감정 예측 함수

In [29]:
import torch

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # GPU에서 동작

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        label = torch.argmax(probs, dim=-1).item()
        confidence = round(probs[0][label].item(), 4)

    return {
        "label": "긍정" if label == 1 else "부정",
        "confidence": confidence
    }

In [30]:
print(predict_sentiment("진짜 너무 맛있고 친절했어요!"))
print(predict_sentiment("별로예요. 다시는 안 가요."))


{'label': '긍정', 'confidence': 0.9975}
{'label': '부정', 'confidence': 0.9985}
